In [1]:
%load_ext autoreload
%autoreload 2


import numpy as np
import rdkit
from rdkit import Chem

import h5py, ast, pickle

# Occupy a GPU for the model to be loaded 
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
# GPU ID, if occupied change to an available GPU ID listed under !nvidia-smi
%env CUDA_VISIBLE_DEVICES=2 

from ddc_pub import ddc_v3 as ddc

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2 # GPU ID, if occupied change to an available GPU ID listed under !nvidia-smi


Using TensorFlow backend.


In [2]:
# Import existing (trained) model
# Ignore warning(s) about training configuration or non-seriazable keyword arguments
model_name = "models/heteroencoder/0.1_512_4_512_l2-1e-7"

model = ddc.DDC(model_name=model_name)

Initializing model in test mode.
Loading model.


/projects/cc/kjmv588/miniconda3/envs/ddc/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
/projects/cc/kjmv588/miniconda3/envs/ddc/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer Decoder_LSTM_0 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'Decoder_State_h_0:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'Decoder_State_c_0:0' shape=(?, 512) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
/projects/cc/kjmv588/miniconda3/envs/ddc/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer Decoder_LSTM_1 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'Decoder_State_h_1:0' shape=(?, 512) dtype=float32>, <tf

Loading finished in 125 seconds.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Encoder_Inputs (InputLayer)     (None, 143, 35)      0                                            
__________________________________________________________________________________________________
mol_to_latent_model (Model)     (None, 512)          3226112     Encoder_Inputs[0][0]             
__________________________________________________________________________________________________
Decoder_Inputs (InputLayer)     (None, 142, 35)      0                                            
__________________________________________________________________________________________________
latent_to_states_model (Model)  [(None, 512), (None, 2101248     mol_to_latent_model[1][0]        
____________________________________________________________________________

In [3]:
# Input SMILES to auto-encode
smiles_in = ['Cc1cccn2c(CN(C)C3CCCc4ccccc43)c(C(=O)N3CCOCC3)nc12',
             'COC(=O)NN=C(c1ccc(O)cc1)C1C(=O)N(C)C(=O)N(C)C1=O',
             'CCc1cc(CC)nc(OCCCn2c3c(c4cc(-c5nc(C)no5)ccc42)CC(F)(F)CC3)n1',
             'Cc1ccc2c(C(=O)Nc3ccccc3)c(SSc3c(C(=O)Nc4ccccc4)c4ccc(C)cc4n3C)n(C)c2c1',
             'Cc1cccc(-c2ccccc2)c1Oc1nc(O)nc(NCc2ccc3occc3c2)n1',
             'Cn1nnnc1SCC(=O)NN=Cc1ccc(Cl)cc1',
             'COc1cccc(NS(=O)(=O)c2ccc(OC)c(OC)c2)c1',
             'COc1ccc(OC)c(S(=O)(=O)n2nc(C)cc2C)c1',
             'NCCCn1cc(C2=C(c3ccncc3)C(=O)NC2=O)c2ccccc21',
             'CN(C)C(=O)N1CCN(C(c2ccc(Cl)cc2)c2cccnc2)CC1']

# MUST convert SMILES to binary mols for the model to accept them (it re-converts them to SMILES internally)
mols_in = [Chem.rdchem.Mol.ToBinary(Chem.MolFromSmiles(smiles)) for smiles in smiles_in]

In [4]:
# Encode the binary mols into their latent representations
latent = model.transform(model.vectorize(mols_in))

In [5]:
# Convert back to SMILES
smiles_out = []
for lat in latent:   
    smiles, _ = model.predict(lat, temp=0)
    smiles_out.append(smiles)

In [6]:
# To compare the results, convert smiles_out to CANONICAL
for idx, smiles in enumerate(smiles_out):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        smiles_out[idx] = Chem.MolToSmiles(mol, canonical=True)
    else:
        smiles_out[idx] = "INVALID"

In [7]:
smiles_in

['Cc1cccn2c(CN(C)C3CCCc4ccccc43)c(C(=O)N3CCOCC3)nc12',
 'COC(=O)NN=C(c1ccc(O)cc1)C1C(=O)N(C)C(=O)N(C)C1=O',
 'CCc1cc(CC)nc(OCCCn2c3c(c4cc(-c5nc(C)no5)ccc42)CC(F)(F)CC3)n1',
 'Cc1ccc2c(C(=O)Nc3ccccc3)c(SSc3c(C(=O)Nc4ccccc4)c4ccc(C)cc4n3C)n(C)c2c1',
 'Cc1cccc(-c2ccccc2)c1Oc1nc(O)nc(NCc2ccc3occc3c2)n1',
 'Cn1nnnc1SCC(=O)NN=Cc1ccc(Cl)cc1',
 'COc1cccc(NS(=O)(=O)c2ccc(OC)c(OC)c2)c1',
 'COc1ccc(OC)c(S(=O)(=O)n2nc(C)cc2C)c1',
 'NCCCn1cc(C2=C(c3ccncc3)C(=O)NC2=O)c2ccccc21',
 'CN(C)C(=O)N1CCN(C(c2ccc(Cl)cc2)c2cccnc2)CC1']

In [8]:
smiles_out

['Cc1cccn2c(CN(C)C3CCCc4ccccc43)c(C(=O)N3CCOCC3)nc12',
 'COC(=O)NN=C1C(=O)N(C)C(=O)N(C)C1=O',
 'CCc1cc(C)nc(OCCCn2c3c(c4cc(-c5nc(CC)no5)ccc42)CCC(F)(F)C3)n1',
 'Cc1ccc2c(C(=O)Nc3ccccc3)c(SSc3c(C(=O)Nc4ccccc4)c4ccc(C)cc4n3C)n(C)c2c1',
 'Cc1cccc(O)c1CNc1nc(Oc2ccc3occc3c2)nc(-c2ccccc2)n1',
 'Cn1nnnc1SCC(=O)NN=Cc1ccc(Cl)cc1',
 'COc1cccc(NS(=O)(=O)c2ccc(OC)c(OC)c2)c1',
 'COc1ccc(OC)c(S(=O)(=O)n2nc(C)cc2C)c1',
 'NCCCn1cc(C2=C(c3ccncc3)C(=O)NC2=O)c2ccccc21',
 'CN(C)C(=O)N1CCN(C(c2ccc(Cl)cc2)c2cccnc2)CC1']

In [9]:
# Validity
out = np.asarray(smiles_out)
valids = len(out[out[:]!="INVALID"])
print("Validity: %.2f" % ( valids / len(out)))

# Reconstructability
print("Reconstructability: %.2f" % ( len(set(smiles_in) & set(smiles_out)) / valids ) )

Validity: 1.00
Reconstructability: 0.70
